In [1]:
import json
import pathlib
import os

In [2]:
from_api_file = 'bioschemastypes_from_API.json'
from_repo_file = 'bioschemastypes_from_REPO.json'

In [3]:
with open(from_api_file,'r') as tempinfile:
    from_api = json.load(tempinfile)

with open(from_repo_file,'r') as tempinfile2:
    from_repo = json.load(tempinfile2)

In [4]:
print(from_api.keys())

dict_keys(['name', 'url', 'source', 'total', 'hits'])


In [5]:
print(from_api['source'].keys())
print(from_api['source']['@context'])

dict_keys(['@context', '@graph'])
{'schema': 'http://schema.org/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#', 'bioschemastypes': 'https://discovery.biothings.io/view/bioschemastypes/', 'bioschemas': 'https://discovery.biothings.io/view/bioschemas/', 'bioschemasdrafts': 'https://discovery.biothings.io/view/bioschemasdrafts/', 'bioschemastypesdrafts': 'https://discovery.biothings.io/view/bioschemastypesdrafts/', 'bioschemasdeprecated': 'https://discovery.biothings.io/view/bioschemasdeprecated/', 'dct': 'http://purl.org/dc/terms/', 'owl': 'http://www.w3.org/2002/07/owl#', 'schema:dateModified': '04/07/2022, 19:26:16'}


In [6]:
print(from_repo.keys())
print(from_repo['@context'])

dict_keys(['@context', '@graph'])
{'schema': 'http://schema.org/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#', 'bioschemastypes': 'https://discovery.biothings.io/view/bioschemastypes/', 'bioschemas': 'https://discovery.biothings.io/view/bioschemas/', 'bioschemasdrafts': 'https://discovery.biothings.io/view/bioschemasdrafts/', 'bioschemastypesdrafts': 'https://discovery.biothings.io/view/bioschemastypesdrafts/', 'bioschemasdeprecated': 'https://discovery.biothings.io/view/bioschemasdeprecated/', 'dct': 'http://purl.org/dc/terms/', 'owl': 'http://www.w3.org/2002/07/owl#', 'schema:dateModified': '04/07/2022, 19:26:16'}


In [13]:
## This script checks to see if the keys for the @context and @graph are the same
def check_context(from_api,from_repo):
    api_context = from_api['source']['@context']
    repo_context = from_repo['@context']
    if api_context.keys() == repo_context.keys():
        isSame = True
    else:
        isSame = False
    return (isSame)

def get_graphs(from_api,from_repo):
    api_graph = from_api['source']['@graph']
    repo_graph = from_repo['@graph']
    api_graphlist = [x['@id'] for x in api_graph]
    repo_graphlist = [x['@id'] for x in repo_graph]
    return(api_graphlist,repo_graphlist)

def check_graph_elements(from_api,from_repo):
    api_graphlist,repo_graphlist = get_graphs(from_api,from_repo)
    api_repo_intersect = [x for x in api_graphlist if x not in repo_graphlist]
    repo_api_intersect = [x for x in repo_graphlist if x not in api_graphlist]
    if len(api_repo_intersect)+len(repo_api_intersect)==0:
        isSameElements = True
    else:
        isSameElements = False
    return (isSameElements)

In [8]:
isSameContext = check_context(from_api,from_repo)
isSameElements = check_graph_elements(from_api,from_repo)
print(isSameContext)
print(isSameElements)

True
True


In [9]:
def dict_compare(d1, d2):
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    added = d1_keys - d2_keys
    removed = d2_keys - d1_keys
    modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
    same = set(o for o in shared_keys if d1[o] == d2[o])
    return added, removed, modified, same

In [16]:
def get_detailed_comparison(from_api,from_repo):
    api_graphlist,repo_graphlist = get_graphs(from_api,from_repo)
    addedlist = []
    removedlist = []
    modifiedlist = []
    idlist = [x['@id'] for x in api_graphlist]
    for i in idlist:
        added, removed, modified, same = dict_compare(from_api['source']['@graph'][i], from_repo['@graph'][i])
        if len(added)>0:
            addedlist.extend(list(added))
        if len(removed)>0:
            removedlist.extend(list(removed))
        if len(modified.keys())>0:
            modifiedlist.append(modified)
    return(addedlist,removedlist,modifiedlist)

In [17]:
addedlist,removedlist,modifiedlist = get_detailed_comparison(from_api,from_repo)
print(addedlist)

TypeError: list indices must be integers or slices, not str